In [ ]:
import numpy as np
import scipy
from scipy.sparse import diags
import matplotlib.pyplot as plt

l = 10          #Length of the beam
l_T = 1         #Length of the region with imposed temperature
T_i = 1         #imposed temperature
A_P = 4         #Area of the parent element
A_C = 0.01      #Area of the child element
sf = A_C / A_P  #Scaling factor
Ncp = 50        #Number of contour levels

ke_p = np.eye(4)*2/3 + np.eye(4, k=1)*(-1/6) + np.eye(4, k=2)*(-1/3) + np.eye(4, k=3)*(-1/6)
de_p = diags(np.diagonal(ke_p), 0).toarray()        # matrix containing only the diagonal entries of ke_p matrix
ke_p = ke_p + np.transpose(ke_p) - de_p             # parent stiffness matrix
                                                    # subtracting de matrix to avoid the doubling of the diagonal entries of ke_p
print (ke_p)
k_c = sf*ke_p     #child stiffness matrix
print (k_c)

# Mesh definition
x=np.linspace(0,10,101)
y=np.linspace(0,1,11)

nodes=np.zeros((1111,2))

k=0
for i in x:
  for j in y:
    nodes[k,0] = i
    nodes[k,1] = j
    k+=1

connectivity = np.zeros((1000,4))

k=0
for i in range(len(x)-1):
  for j in range(len(y)-1):
    connectivity[k,0] = (1+j)+(i*11)
    connectivity[k,1] = (2+j)+(i*11)
    connectivity[k,2] = (12+j)+(i*11)
    connectivity[k,3] = (13+j)+(i*11)
    k+=1

print (nodes)
print (connectivity)

nodes_int = nodes.astype(int)
connectivity_int = connectivity.astype(int)
connectivity -= 1
connectivity_int -= 1
nnodes = nodes.shape[0]
nel = connectivity.shape[0]

print ('number of nodes = ', nnodes)
print ('number of elements = ', nel)

k = np.zeros((nnodes, nnodes))    # initiating global stiffness matrix as zero matrix
for element in range(nel):
  for i in range(4):    # line number of k_c
    for j in range(4):  # column number of k_c
      k[connectivity_int[element,i],connectivity_int[element,j]] += k_c[i,j]

plt.show(plt.spy(k))     # plot skyline of global matrix

#an array of unknown size to contain the node IDs for which T=0
nent = []        
for i in range(nnodes):
  if nodes[i,0] == 0:
    nent.append(i)
print ('Node IDs with T=0 is ', nent)

#an array of unknown size to contain the node IDs with imposed temperature
oent = []
for i in range(nnodes):
  if nodes[i,0] >= (l-l_T) and nodes[i,0] <= l and nodes[i,1] == 0:
      oent.append(i)

print ('Node IDs with imposed temperature is ', oent)

#Swapping the rows and columns of k matrix corresponding to T=0
for i in range (len(nent)):
  k[[i, nent[i]],:] = k[[nent[i], i],:]
  k[:,[i, nent[i]]] = k[:,[nent[i], i]]

#Swapping the rows and columns of k matrix corresponding to T=1
for i in range (len(oent)):  
  k[[i+len(nent), oent[i]],:] = k[[oent[i], i+len(nent)],:]
  k[:,[i+len(nent), oent[i]]] = k[:,[oent[i], i+len(nent)]] 

#Deleting the rows and columns of k matrix corresponding to T=0
for i in range (len(nent)):
  k = np.delete(np.delete(k, 0, 0), 0, 1)
                                        
#Deleting the rows of k matrix corresponding to T=1
for i in range (len(oent)):
  k = np.delete(k, 0, 0)

#Making the K* matrix corresponding to T_i
k_st = np.zeros((k.shape[0],len(oent)))
for j in range(len(oent)):
  for i in range(k.shape[0]):
    k_st[i,j] = k[i,j]

#Deleting the columns of k matrix corresponding to T_i
for i in range (len(oent)):
  k = np.delete(k, 0, 1) 

T_imposed = T_i*np.ones(len(oent))
A = np.linalg.inv(k)
B = np.dot(k_st,T_imposed.transpose())
Temp = -np.dot(A,B)

#Creating the complete temperature matrix
Temperature = np.zeros((nnodes, 1))
for i in range (len(nent)):
  Temperature[i] = 0

for i in range(len(nent), len(nent)+len(oent)):
  Temperature[i] = T_i

for i in range(len(nent)+len(oent), nnodes):
  Temperature[i] = Temp[i-len(nent)-len(oent)]

#Swapping the rows of T matrix to return the nodes to original position in the matrix
for i in range (len(nent)):
  Temperature[[i, nent[i]]] = Temperature[[nent[i], i]]

for i in range (len(oent)):
  Temperature[[i+len(nent), oent[i]]] = Temperature[[oent[i], i+len(nent)]]

#Plotting contour
f = plt.figure()
f.set_figwidth(20)
f.set_figheight(2)

plt.tricontourf(nodes[:,0],nodes[:,1],Temperature[:,0], Ncp)
plt.colorbar()
plt.show()